In [1]:
import torch
import warnings
warnings.filterwarnings('ignore')

# 计算图
## 计算图

In [2]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward()
print(w.grad)

tensor([5.])


1. 叶子节点这个属性(还记得张量的属性里面有一个 is_leaf 吗）: **叶子节点：用户创建的节点**， 比如上面的 x 和 w。叶子节点是非常关键的，在上面的正向计算和反向计算中，其实都是依赖于我们叶子节点进行计算的。is_leaf: 指示张量是否是叶子节点。

为什么要设置叶子节点的这个概念的？主要是为了节省内存，因为我们在反向传播完了之后，非叶子节点的梯度是默认被释放掉的。我们可以根据上面的那个计算过程，来看看 w，x, a, b, y 的 is_leaf 属性，和它们各自的梯度情况：

In [3]:
# 查看叶子结点
print("is_leaf:\n", w.is_leaf, x.is_leaf, a.is_leaf, b.is_leaf, y.is_leaf)
# 查看梯度， 默认是只保留叶子节点的梯度的
print("gradient:\n", w.grad, x.grad, a.grad, b.grad, y.grad)

is_leaf:
 True True False False False
gradient:
 tensor([5.]) tensor([2.]) None None None


In [4]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
a.retain_grad()
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward()
#查看梯度， 默认是只保留叶子节点的梯度的
print("gradient:\n", w.grad, x.grad, a.grad, b.grad, y.grad)

gradient:
 tensor([5.]) tensor([2.]) tensor([2.]) None None


2. grad_fn：记录创建该张量时所用的方法（函数），记录这个方法主要「用于梯度的求导」。要不然怎么知道具体是啥运算？

In [5]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
a.retain_grad()
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward()

# 查看 grad_fn   这个表示怎么得到的
print("grad_fn:\n", w.grad_fn, x.grad_fn, a.grad_fn, b.grad_fn, y.grad_fn)

grad_fn:
 None None <AddBackward0 object at 0x000001AB1EA90E10> <AddBackward0 object at 0x000001AB1EA90EB8> <MulBackward0 object at 0x000001AB1EA90F60>


## 动态图
根据计算图的搭建方式，可以将计算图分为动态图和静态图。
- 静态图：先搭建图，后运算。高效，不灵活（TensorFlow）
- 动态图：运算与搭建同时进行。灵活，易调节（Pytorch）

In [6]:
# import tensorflow as tf
# # 声明两个常量
# w = tf.constant(1.)
# x = tf.constant(2.)

# # 搭建静态图
# a = tf.add(w, x)
# b = tf.add(w, 1)
# y = tf.multiply(a, b)

# # 这时候还没开始计算
# print(y)   # Tensor("Mul_4:0", shape=(), dtype=float32)， 只是计算图的一个节点

# with tf.Session() as sess:  
#     print(sess.run(y))   # 这里才开始进行计算， 6.0

In [7]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)
print(y)    # tensor([6.], grad_fn=<MulBackward0>)

tensor([6.], grad_fn=<MulBackward0>)


# 自动求导机制